# Fleming Pipeline - ipython notebook version

Note on astrometry.net keys:
[Go to the website](http://nova.astrometry.net/api_help) and sign in and follow the instructions.

Key shouldn't matter if not `force_upload==False`
~~Need to add your own key to `~/.astroquery/config/astroquery.cfg` and `pipeline/astrometry_api_key.txt`~~

In [ ]:
## Imports
import Reducer
import ShiftFinder
import FluxFinder
import DataAnalyser
import Constants
import Utilities
import MovingObjectFinder
import StreakFinder
import Cataloguer

from datetime import datetime
import os
import importlib

#from astroquery import log
#log.setLevel("TRACE")


In [ ]:
# REMOVE ME WHEN NOT USING DARK MODE
import matplotlib as mpl
COLOR = "white"
#OLOR = "black"
mpl.rcParams['text.color'] = COLOR
mpl.rcParams['axes.labelcolor'] = COLOR
mpl.rcParams['xtick.color'] = COLOR
mpl.rcParams['ytick.color'] = COLOR

In [ ]:
config = Constants.Config(
    #image_subdir = "../images/0121",
    image_dir = "/home/callum/mnt/data/jgtdata/l137_0/0121",
    image_prefix = "l137_0",
    bias_prefix = "bias_shutter",
    has_sets = True,
    set_size = 50,
    n_sets = 7
)

In [ ]:
r = Reducer.Reducer(config, "No filter") ## Only "No filter" for Trius
r.reduce(skip_existing=True)
#r.reduce(skip_existing=False)

In [ ]:
c = Cataloguer.Cataloguer(config)

catalogue_image = os.path.join(config.image_dir, config.image_format_str.format(1, 1))

n_sources = c.generate_catalogue(catalogue_image, solve=False)
c.generate_image_times()

In [ ]:
sf = ShiftFinder.ShiftFinder(config, n_sources)

sf.generate_shifts()

In [ ]:
ff = FluxFinder.FluxFinder(config, n_sources)

ff.make_light_curves()

In [ ]:
da = DataAnalyser.DataAnalyser(config)

print("[Main] Creating average light curve")
da.create_avg_curve()
ff.plot_avg_light_curve(config.avg_curve_path, show=True)

print("[Main] Adjusting")
#ff.create_adjusted_light_curves()

In [ ]:
print("[Main] Plotting variable light curves")
variable_ids = da.get_variables()
print(variable_ids)
ff.plot_all_light_curves(variable_ids, adjusted=True, show=True)

In [ ]:
da.plot_means_and_stds(adjusted=True)
da.output_results()
da.create_thumbnails(ff)

## Debug land

Use at your own peril

In [ ]:
## Imports
import Reducer
import ShiftFinder
import FluxFinder
import DataAnalyser
import Constants
import Utilities
import MovingObjectFinder
import StreakFinder
import Cataloguer

from datetime import datetime
import os
import importlib

In [ ]:
debug_config = Constants.Config(
    #image_subdir = "../images/0121",
    image_dir = "/home/callum/mnt/data/jgtdata/l137_0/0121",
    image_prefix = "l137_0",
    has_sets = True,
    set_size = 50,
    n_sets = 7
)

In [ ]:
Reducer = importlib.reload(Reducer)
ShiftFinder = importlib.reload(ShiftFinder)
FluxFinder = importlib.reload(FluxFinder)
DataAnalyser = importlib.reload(DataAnalyser)
Utilities = importlib.reload(Utilities)
Cataloguer = importlib.reload(Cataloguer)
Constants = importlib.reload(Constants)

In [ ]:
Cataloguer = importlib.reload(Cataloguer)

c = Cataloguer.Cataloguer(debug_config)

catalogue_image = os.path.join(debug_config.image_dir,
        debug_config.image_format_str.format(1, 1))

n_sources = c.generate_catalogue(catalogue_image, solve=False)
c.generate_image_times()

In [ ]:
FluxFinder = importlib.reload(FluxFinder)
ff = FluxFinder.FluxFinder(debug_config, n_sources)
#ff = FluxFinder.FluxFinder(config, n_sources)

#ff.make_light_curves()

In [ ]:
DataAnalyser = importlib.reload(DataAnalyser)
da = DataAnalyser.DataAnalyser(debug_config)

da.create_avg_curve()

In [ ]:
FluxFinder = importlib.reload(FluxFinder)
ff = FluxFinder.FluxFinder(debug_config, n_sources)

ff.plot_avg_light_curve(debug_config.avg_curve_path, show=True)

ff.create_adjusted_light_curves()
ff.plot_all_light_curves(variable_ids, adjusted=True, show=True)

In [ ]:
DataAnalyser = importlib.reload(DataAnalyser)

da.get_variables()
